In [1]:
import os, sys
import numpy as np
import pandas as pd
import clickhouse_connect
import matplotlib.pyplot as plt
from collections import defaultdict

import re

In [2]:
host = 'mini.local'
username = 'admin'
port = 8123
client = clickhouse_connect.get_client(host=host, username=username, port=port)

In [3]:
stable_coins = [
    "USDCUSDT",
    "BUSDUSDT",
    "USTUSDT",
    "TUSDUSDT",
    "UNIUSDT",
    "LUNAUSDT",
    "FDUSDUSDT",
]

In [4]:
symbols = client.command('show tables from binance_hist_spot').split('\n')

In [5]:
N = 20
start_year, end_year = 2021, 2024

data = {}
for symbol in symbols:
    if symbol.upper() in stable_coins: continue
    print(symbol)
    try:
        df = client.query_df(f"select * from binance_hist_spot.{symbol} where open_time >= '{start_year-1}-12-01' and close_time < '{end_year + 1}-01-01'")
    except Exception as e:
        print(f'Could not query {symbol}: {e}')
        continue
    data[symbol] = df

1inchdownusdt
1inchupusdt
1inchusdt
aavedownusdt
aaveupusdt
aaveusdt
acausdt
achusdt
acmusdt
adadownusdt
adaupusdt
adausdt
adxusdt
aergousdt
agixusdt
agldusdt
aionusdt
akrousdt
alcxusdt
algousdt
aliceusdt
alpacausdt
alphausdt
alpineusdt
ambusdt
ampusdt
ancusdt
ankrusdt
antusdt
anyusdt
apeusdt
api3usdt
aptusdt
arbusdt
ardrusdt
arkmusdt
arkusdt
arpausdt
arusdt
asrusdt
astrusdt
astusdt
atausdt
atmusdt
atomusdt
auctionusdt
audiousdt
audusdt
autousdt
avausdt
avaxusdt
axsusdt
badgerusdt
bakeusdt
balusdt
bandusdt
barusdt
batusdt
bccusdt
bchabcusdt
bchdownusdt
bchsvusdt
bchupusdt
bchusdt
beamusdt
bearusdt
belusdt
betausdt
bethusdt
bicousdt
bifiusdt
bkrwusdt
blzusdt
bnbbearusdt
bnbbullusdt
bnbdownusdt
bnbupusdt
bnbusdt
bntusdt
bnxusdt
bondusdt
bswusdt
btcdownusdt
btcstusdt
btcupusdt
btcusdt
btgusdt
btsusdt
bttcusdt
bttusdt
bullusdt
burgerusdt
bzrxusdt
c98usdt
cakeusdt
celousdt
celrusdt
cfxusdt
chessusdt
chrusdt
chzusdt
cityusdt
ckbusdt
clvusdt
cocosusdt
combousdt
compusdt
cosusdt
cotiusdt
cream

In [6]:
monthly_data = defaultdict(lambda: defaultdict(pd.DataFrame))

for symbol, df in data.items():
    if df.empty: continue
    print(f'No data from {symbol}')
    for year in range(start_year - 1, end_year + 1):
        for month in range(1, 13):
            if year == start_year - 1 and month < 12: continue
            df_month = df.loc[(df['open_time'].dt.year == year) & (df['open_time'].dt.month == month)]
            if df_month.empty: continue
            monthly_data[f'{year:04}-{month:02}'][symbol.upper()] = df_month

No data from 1inchdownusdt
No data from 1inchupusdt
No data from 1inchusdt
No data from aavedownusdt
No data from aaveupusdt
No data from aaveusdt
No data from acausdt
No data from achusdt
No data from acmusdt
No data from adadownusdt
No data from adaupusdt
No data from adausdt
No data from adxusdt
No data from aergousdt
No data from agixusdt
No data from agldusdt
No data from aionusdt
No data from akrousdt
No data from alcxusdt
No data from algousdt
No data from aliceusdt
No data from alpacausdt
No data from alphausdt
No data from alpineusdt
No data from ambusdt
No data from ampusdt
No data from ancusdt
No data from ankrusdt
No data from antusdt
No data from anyusdt
No data from apeusdt
No data from api3usdt
No data from aptusdt
No data from arbusdt
No data from ardrusdt
No data from arkmusdt
No data from arkusdt
No data from arpausdt
No data from arusdt
No data from asrusdt
No data from astrusdt
No data from astusdt
No data from atausdt
No data from atmusdt
No data from atomusdt
No d

In [ ]:
volumes_by_month = defaultdict(list)
data = monthly_data

for ym in data:
    print(f'Processing {ym}')
    df_btc = data[ym]['BTCUSDT'].set_index('open_time')
    df_eth = data[ym]['ETHUSDT'].set_index('open_time')
    df_daily_volume_btc = df_btc["taker_buy_quote_asset_volume"].resample("1D", closed="left", label="left").sum()
    df_daily_volume_eth = df_eth["taker_buy_quote_asset_volume"].resample("1D", closed="left", label="left").sum()
    df_prc_btc = df_btc['open'].resample('1h', closed='left', label='left').first().ffill().bfill()
    df_prc_eth = df_eth['open'].resample('1h', closed='left', label='left').first().ffill().bfill()
    df_ret_btc = np.log(df_prc_btc).diff()
    df_ret_eth = np.log(df_prc_eth).diff()

    for sym in data[ym]:
        if sym in ['BTCUSDT', 'ETHUSDT']: continue
        df = data[ym][sym].set_index('open_time')
        df_daily_volume = df["taker_buy_quote_asset_volume"].resample("1D", closed="left", label="left").sum()
        df_prc = df['open'].resample('1h', closed='left', label='left').first().ffill().bfill()
        df_ret = np.log(df_prc).diff()
        df_ret_tmp = pd.DataFrame({
            'BTCUSDT': df_ret_btc,
            'ETHUSDT': df_ret_eth,
            sym: df_ret}).fillna(0)
        btc_corr = df_ret_tmp.corr().iloc[0, 2]
        eth_corr = df_ret_tmp.corr().iloc[1, 2]

        vol_btc = df_daily_volume.median() / df_daily_volume_btc.median()
        vol_eth = df_daily_volume.median() / df_daily_volume_eth.median()
        volumes_by_month[ym].append((-df_daily_volume.median(), vol_btc, vol_eth, sym[:-4], btc_corr, eth_corr))



In [ ]:
months = sorted(volumes_by_month.keys())
results = defaultdict(list)

In [ ]:
all_symbols = set()
                    
for i in range(0, len(months)):
    # univ_month = sorted(volumes_by_month[months[i - 1]])
    univ_month = sorted(volumes_by_month[months[i]])
    vols_b  = [x[1] for x in univ_month]
    vols_e  = [x[2] for x in univ_month]
    symbols = [x[3] for x in univ_month]
    btc_cor = [x[4] for x in univ_month]
    eth_cor = [x[5] for x in univ_month]
    univ = []           
    for j in range(len(univ_month)):
        if j == N: break
        univ += f'{symbols[j]}|{vols_b[j]:.2f}|{vols_e[j]:.2f}|{btc_cor[j]:.2f}|{eth_cor[j]:.2f}',
                        
    if len(univ) < N: univ += [''] * (N - len(univ))

    all_symbols |= set(symbols[:N])
    results[months[i] + "-01"] = univ

df = pd.DataFrame(results) 
df.to_csv("univ_by_month_taker.csv", index=False)

with open("univ.csv", 'w') as f:
    for sym in all_symbols:
        if not sym == "":
            f.write(f"{sym}\n")


In [ ]:
df

,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01,...,2024-02-01,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01
0,XRP|0.23|0.72|0.44|0.52,BUSD|0.10|0.17|-0.35|-0.35,BNB|0.22|0.46|0.52|0.54,BNB|0.23|0.63|0.69|0.69,XRP|0.62|1.14|0.59|0.58,DOGE|0.63|0.60|0.56|0.61,BUSD|0.27|0.46|-0.00|-0.01,BUSD|0.37|0.43|0.01|0.01,BUSD|0.37|0.45|0.01|0.01,SOL|0.48|0.66|0.58|0.63,...,FDUSD|0.49|0.65|0.09|0.06,FDUSD|0.96|1.77|nan|nan,FDUSD|0.36|0.74|0.22|0.21,USDC|0.42|0.73|0.02|0.12,USDC|0.52|0.89|0.09|0.21,USDC|0.51|0.94|0.08|0.17,USDC|0.47|0.86|-0.05|-0.01,USDC|0.46|1.07|-0.04|0.02,USDC|0.48|1.08|-0.05|0.00,USDC|0.45|0.81|-0.22|-0.07
1,LTC|0.12|0.36|0.77|0.81,DOT|0.09|0.15|0.53|0.56,BUSD|0.20|0.42|-0.18|-0.19,BUSD|0.20|0.53|-0.09|-0.09,BNB|0.55|1.02|0.62|0.67,BUSD|0.43|0.41|-0.03|-0.17,BNB|0.24|0.42|0.88|0.89,AXS|0.26|0.30|0.31|0.38,ADA|0.31|0.37|0.62|0.66,BUSD|0.44|0.61|0.09|0.08,...,USDC|0.33|0.44|-0.07|0.02,SOL|0.32|0.58|0.69|0.71,USDC|0.35|0.71|0.04|0.11,SOL|0.35|0.61|0.73|0.66,SOL|0.26|0.44|0.72|0.75,SOL|0.35|0.64|0.78|0.76,SOL|0.33|0.61|0.83|0.76,FDUSD|0.26|0.61|0.16|0.17,SOL|0.30|0.68|0.81|0.79,DOGE|0.45|0.80|0.68|0.60
2,GRT|0.09|0.28|nan|nan,LTC|0.08|0.13|0.77|0.81,ADA|0.17|0.36|0.47|0.56,ADA|0.12|0.33|0.47|0.45,BUSD|0.42|0.77|0.22|0.08,BNB|0.41|0.39|0.87|0.89,MATIC|0.16|0.28|0.79|0.80,BNB|0.26|0.30|0.83|0.86,XRP|0.26|0.32|0.70|0.70,ADA|0.27|0.37|0.74|0.77,...,SOL|0.29|0.38|0.61|0.68,USDC|0.28|0.51|-0.03|0.03,SOL|0.30|0.61|0.83|0.84,PEPE|0.27|0.47|0.54|0.57,FDUSD|0.24|0.40|0.20|0.24,FDUSD|0.17|0.33|nan|nan,FDUSD|0.29|0.53|0.25|0.32,SOL|0.26|0.61|0.81|0.81,PEPE|0.14|0.32|0.78|0.78,SOL|0.34|0.62|0.70|0.71
3,BUSD|0.09|0.27|0.05|0.05,XRP|0.07|0.11|0.42|0.43,XRP|0.15|0.31|0.41|0.41,CHZ|0.11|0.29|0.17|0.17,DOGE|0.40|0.73|0.27|0.26,XRP|0.34|0.32|0.81|0.81,DOGE|0.16|0.27|0.68|0.67,DOGE|0.15|0.18|0.72|0.71,BNB|0.25|0.30|0.80|0.80,BNB|0.24|0.33|0.83|0.85,...,BNB|0.13|0.17|0.51|0.61,PEPE|0.23|0.42|nan|nan,DOGE|0.14|0.29|0.83|0.83,FDUSD|0.22|0.39|0.16|0.14,PEPE|0.23|0.39|0.62|0.72,PEPE|0.17|0.31|nan|nan,PEPE|0.14|0.26|0.79|0.81,PEPE|0.10|0.23|0.72|0.75,SUI|0.11|0.26|0.53|0.55,XRP|0.25|0.45|0.40|0.41
4,BCH|0.05|0.15|0.69|0.75,LINK|0.07|0.11|0.60|0.70,DOT|0.12|0.25|0.61|0.68,XRP|0.08|0.22|0.54|0.55,WIN|0.17|0.32|0.37|0.38,ADA|0.30|0.28|0.77|0.79,ADA|0.15|0.26|0.84|0.85,XRP|0.14|0.16|0.75|0.75,DOGE|0.24|0.29|0.72|0.71,DOT|0.22|0.30|0.76|0.80,...,XRP|0.12|0.16|0.60|0.62,BNB|0.19|0.35|0.66|0.70,BNB|0.13|0.27|0.80|0.80,DOGE|0.12|0.21|0.69|0.63,BNB|0.09|0.16|0.60|0.63,XRP|0.12|0.22|0.46|0.50,XRP|0.09|0.16|0.64|0.62,SUI|0.09|0.20|0.50|0.49,DOGE|0.10|0.24|0.62|0.62,FDUSD|0.23|0.40|0.32|0.23
5,LINK|0.04|0.13|0.57|0.80,ADA|0.06|0.10|0.63|0.68,DOGE|0.11|0.24|0.24|0.27,DOT|0.08|0.21|0.68|0.67,ADA|0.15|0.28|0.75|0.71,MATIC|0.24|0.23|0.63|0.66,XRP|0.13|0.23|0.82|0.84,ADA|0.11|0.13|0.85|0.85,DOT|0.15|0.18|0.72|0.75,AVAX|0.21|0.29|0.60|0.64,...,ARB|0.07|0.09|0.64|0.84,DOGE|0.18|0.33|0.67|0.67,PEPE|0.12|0.23|0.76|0.79,BNB|0.09|0.16|0.72|0.68,XRP|0.08|0.13|0.57|0.61,BNB|0.07|0.14|0.79|0.81,BNB|0.09|0.16|0.87|0.84,BNB|0.09|0.20|0.77|0.77,BNB|0.07|0.17|0.79|0.79,PEPE|0.18|0.32|0.57|0.65
6,BNB|0.04|0.12|0.59|0.66,BCH|0.04|0.07|0.68|0.73,LTC|0.08|0.18|0.75|0.77,ENJ|0.05|0.13|0.27|0.25,BTT|0.15|0.27|0.42|0.44,DOT|0.17|0.17|0.79|0.81,DOT|0.09|0.16|0.81|0.83,DOT|0.10|0.11|0.80|0.83,SOL|0.13|0.16|0.46|0.49,XRP|0.20|0.28|0.78|0.80,...,LINK|0.06|0.08|0.54|0.64,SHIB|0.13|0.25|0.54|0.53,XRP|0.09|0.19|0.75|0.78,XRP|0.08|0.15|0.60|0.60,PEOPLE|0.08|0.13|0.25|0.29,DOGE|0.05|0.10|0.75|0.78,DOGE|0.04|0.07|0.87|0.86,XRP|0.07|0.17|0.49|0.53,XRP|0.07|0.16|0.54|0.60,BNB|0.11|0.20|0.59|0.69
7,ADA|0.04|0.12|0.65|0.76,XLM|0.04|0.06|0.41|0.45,EOS|0.05|0.11|0.58|0.66,LTC|0.05|0.13|0.77|0.78,VET|0.14|0.25|0.58|0.58,ETC|0.17|0.16|0.65|0.70,SOL|0.06|0.10|0.74|0.76,MATIC|0.09|0.11|0.76|0.80,AXS|0.11|0.13|0.29|0.32,FTM|0.17|0.23|0.58|0.65,...,AVAX|0.05|0.07|0.64|0.72,FLOKI|0.12|0.22|nan|